In [1]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
%matplotlib inline

In [2]:
def extract_tensorboard_data(log_path):
    """从TensorBoard日志文件中提取数据"""
    # 初始化事件累加器
    ea = event_accumulator.EventAccumulator(log_path,
        size_guidance={  # 定义加载的最大事件数量
            event_accumulator.SCALARS: 0,  # 加载所有标量数据
        })
    ea.Reload()  # 加载数据
    
    # 获取所有标量指标的标签
    tags = ea.Tags()['scalars']
    
    # 存储所有指标的数据
    dataframes = {}
    for tag in tags:
        # 获取该指标的所有事件
        events = ea.Scalars(tag)
        
        # 转换为DataFrame
        df = pd.DataFrame(events)
        # 重命名列名使其更直观
        df.columns = ['wall_time', 'step', 'value']
        
        dataframes[tag] = df
    
    return dataframes, tags

In [3]:
def create_combined_dataframe(dataframes):
    """将所有指标数据合并为一个DataFrame，保持原始标签名称"""
    # 创建基础DataFrame，使用第一个指标的step作为索引
    first_key = list(dataframes.keys())[0]
    combined_df = pd.DataFrame(index=dataframes[first_key]['step'])
    
    # 遍历所有指标，将它们的值添加到合并的DataFrame中
    for tag, df in dataframes.items():
        # 保持原始标签名称
        combined_df[tag] = df.set_index('step')['value']
    
    # 重置索引，将step变成一个普通列
    combined_df.reset_index(inplace=True)
    # 重命名'index'列为'Unnamed: 0'以匹配目标格式
    combined_df.rename(columns={'index': 'Unnamed: 0'}, inplace=True)
    
    return combined_df

In [4]:
# 读取tensorboard日志文件
log_path = "./data/stokcs/PPO_7/events.out.tfevents.1733476158.andy-X570-I.26362.0"
dataframes, tags = extract_tensorboard_data(log_path)

# 打印所有可用的指标
print("Available metrics:")
for tag in tags:
    print(f"- {tag}")

# 创建合并的DataFrame并保持原始标签名称
combined_df = create_combined_dataframe(dataframes)

# 保存为CSV文件到当前目录
output_path = './stocks_trained_log.csv'
combined_df.to_csv(output_path, index=False)

# 打印保存位置和数据预览
print(f"\n文件已保存到当前目录: {output_path}")
print("\n数据预览:")
print(combined_df.head())

Available metrics:
- rollout/ep_len_mean
- rollout/ep_rew_mean
- time/fps
- train/approx_kl
- train/clip_fraction
- train/clip_range
- train/entropy_loss
- train/explained_variance
- train/learning_rate
- train/loss
- train/policy_gradient_loss
- train/std
- train/value_loss

文件已保存到当前目录: ./stocks_trained_log.csv

数据预览:
    step  rollout/ep_len_mean  rollout/ep_rew_mean  time/fps  train/approx_kl  \
0   2048                128.0            -0.350610     555.0              NaN   
1   4096                128.0            -0.781948     532.0         0.006492   
2   6144                128.0            -0.696733     527.0         0.009861   
3   8192                128.0            -0.791840     523.0         0.011337   
4  10240                128.0            -0.764007     524.0         0.014558   

   train/clip_fraction  train/clip_range  train/entropy_loss  \
0                  NaN               NaN                 NaN   
1             0.060156               0.2          -15.601842   
